In [6]:
import os
import json
from IPython.display import display, Image

In [4]:
saved_tweets_dir = '/media/zhaomin/Zhaomin_SSD/project_repo/emnlp2021/saved_tweets_original'
events_noinclude = ['valentinesday_2019-02-14_2019-02-14',
                    'valentinesday_2020-02-14_2020-02-14',
                    'newyearseve_2019-12-31_2019-12-31',
                    'zip_files']

num_context = 6
for event in os.listdir(saved_tweets_dir):
    if event not in events_noinclude:
        event_path = os.path.join(saved_tweets_dir, event)
        event_screenshot_path = os.path.join(event_path, f'screenshots_{event}')
        instances = os.listdir(event_screenshot_path)
        for instance in instances:
            instance_path = os.path.join(event_screenshot_path, instance)
            print(os.listdir(instance_path))
            break
        break

['1204849482099245057.png', '1204858122583658496.png', '1204861925139587072.png', '1204863934005538816.png', '1204868504824012800.png', '1204874073169371138.png', '1204881260809728001.png', '1204891051271446528.png', '1204894986535424000.png', '1204898656714776576.png', '1204909058815336449.png', '1204916328915714048.png', '1204921141070094336.png', '1204921324851871744.png', '1204925338947596289.png', '1204929617003470848.png', '1204936063531323392.png', '1204939934274084865.png', '1204942957792632832.png', '1204949434662080513.png', 'anchor_1204903191986548829.png', 'anchor_1204903191986548829_noimage.png', 'anchor_1204903191986548829_notext.png']


In [ ]:
onlytweets_dir = '/home/zhaomin/Documents/saved_tweets/'
instances = []
num_context = 6
for event in os.listdir(onlytweets_dir):
    event_path = os.path.join(onlytweets_dir, event)
    days = os.listdir(event_path)
    for day in days:
        day_path = os.path.join(event_path, day)
        day_instances = os.listdir(day_path)
        for day_instance in day_instances:
            day_instance_path = os.path.join(day_path, day_instance)
            
            # load the 20 context tweets
            tweets = []
            filenames = os.listdir(day_instance_path)
            json_filenames = [x for x in filenames if x.endswith(".json") and not x.startswith("anchor")]
            for json_filename in json_filenames:
                json_filepath = os.path.join(day_instance_path, json_filename)
                with open(json_filepath, 'r') as jsonfile:
                    tweet = json.load(jsonfile)
                    tweets.append(tweet)
            tweets = sorted(tweets, key = lambda x: x['created_at'])
            
            # load the anchor tweet
            anchor_filename = f"anchor_{day_instance}.json"
            anchor_filepath = os.path.join(day_instance_path, anchor_filename)
            with open(anchor_filepath, 'r') as anchor_jsonfile:
                anchor_tweet = json.load(anchor_jsonfile)
            
            # only keep part of the context tweets
            tweets = tweets[10-num_context:10] + [anchor_tweet] + tweets[10:10+num_context]
            instances.append(tweets)
#             break
#         break
#     break

In [24]:
len(instances)

10098

In [14]:
a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
a[5-3:5+3]

[2, 3, 4, 5, 6, 7]

In [16]:
tweets[0]

{'created_at': 'Mon Jun 10 02:56:52 +0000 2019',
 'id': 1137916511123857408,
 'id_str': '1137916511123857408',
 'full_text': 'What does pollution, trash, and plastic all have in common? They all our a danger to our ocean.\xa0\nImagine a world where there were no plastic straws and plastic bags! Beautiful! 😍',
 'truncated': False,
 'display_text_range': [0, 178],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1136685248295706625,
  'id_str': '1136685248295706625',
  'name': 'Protect and Save our Oceans!',
  'screen_name': 'EndDirtyOceans',
  'location': 'Philadelphia, PA',
  'description': 'The ocean is being destroyed and we NEED to make a change NOW! Here to raise more awareness!🌊💪🐠',
  'ur